In [1]:
pip install flask

Note: you may need to restart the kernel to use updated packages.


In [2]:
import ast
import socket
import time
import json
import requests
from http.server import BaseHTTPRequestHandler, HTTPServer
from flask import Flask, request, jsonify

app = Flask(__name__)
users = {}
db_ip = 'http://example.com/api/db'
    
#обработчик запросов от юзера
@app.route('/api/user', methods=['POST'])
def proceed_user_data():
    try:
        user_data = request.json
        login = user_data["login"]
        password = user_data["password"]

        u_id = find_user_id(login, password)
        if u_id:
            # Пользователь найден, проверим пароль
            if users[u_id]["password"] == password:
                # Пароль верный, обновляем данные пользователя
                users[u_id]["last_task_id"] += 1
                users[u_id]["data"] = user_data["data"]
                users[u_id]["function"] = user_data["function"]

                # Отправляем данные в БД
                send_to_db(json.dumps({
                    "user_id": u_id,
                    "last_task_id": users[u_id]["last_task_id"],
                    "data": user_data["data"],
                    "function": user_data["function"]
                }))

                return jsonify({
                    "user_id": u_id,
                    "task_id": users[u_id]["last_task_id"]
                }), 200
            else:
                # Пароль неверный
                return jsonify({"error": "Invalid password"}), 403
        else:
            # Пользователь не найден, создаем нового пользователя
            u_id = hash(login + password)
            users[u_id] = {
                "login": login,
                "password": password,
                "last_task_id": 1,
                "data": user_data["data"],
                "function": user_data["function"]
            }

            return jsonify({
                'user_id': u_id,
                'task_id': users[u_id]["last_task_id"]
            }), 200

        # Отправляем данные в БД
        send_to_db(json.dumps({
            "user_id": u_id,
            "last_task_id": 1,
            "data": user_data["data"],
            "function": user_data["function"]
        }))
        
    except Exception as e:
        app.logger.error(f'Error: {e}')
        return jsonify({"error": str(e)}), 500


#поиск юзер айди по логину и паролю  
def find_user_id(login, password):
    for u_id, user_info in users.items():
        if user_info["login"] == login and user_info["password"] == password:
            return u_id


@app.route('/api/data_base', methods=['GET'])
def user_data_to_db():
    try:
        # Возвращаем все данные пользователей
        return jsonify(users), 200

    except Exception as e:
        return jsonify({"error": str(e)}), 500



def send_to_db(data):
    # Отправка JSON-данных в БД
    headers = {'Content-Type': 'application/json'}
    response = requests.post(bd_ip, data=json.dumps(data), headers=headers)
    if response.status_code == 200:
        print("Данные отправлены на сервер успешно.")
    else:
        print(f"Не удалось отправить данные в базу данных. Код состояния: {response.status_code}")



def get_from_db():
    # Получение данных из БД
    response = requests.get(bd_ip)
    if response.status_code == 200:
        data = response.json()
        return data
    else:
        print(f"Не удалось получить данные из базы данных. Код состояния: {response.status_code}")


#общение с нодой  
@app.route('/api/node', methods=['POST'])
def get_array():
    try:
        # Получаем данные из запроса (начало и конец массива)
        data = request.json

        # Проверяем, что данные переданы
        if not master_data:
            return jsonify({"error": "No data provided"}), 400

        # Проверяем, что все необходимые поля присутствуют
        required_fields = ['user_id', 'task_id', 'start', 'stop']
        for field in required_fields:
            if field not in master_data:
                return jsonify({"error": f"Missing field: {field}"}), 400
            else:
                send_to_db(json.dumps(jsonify(data)))  #отправляем в БД начало и конец массива
                array_info = None
                while (array_info == None):  #ждем ответа от БД  
                    array_info = get_from_db()
                return array_info  #возвращаем запрошенный диапазон ноде  

        # Возвращаем успешный статус
        return jsonify({"status": "success", "message": "Data received"}), 200

    except Exception as e:
        return jsonify({"error": str(e)}), 500



#     elif msg_dict['msg_type']=='MP_master_node':
#         'у нас спрашивают какое кол-во строк(сколько всего нужно обработать), айди (юзера), надо ответить. Мы идем к мастер н, что им надо от нас, делаем ответы на то что хотят'
#     elif payload.get('msg_type')=='db':
#         return




# def send_message_mp(self,u_id,t_id,msg):
# 'возвращаем Message_Processor что-то там, что потребовал'

if __name__ == "__main__":
    app.run(host='0.0.0.0', port=5050)



 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5050
 * Running on http://192.168.0.92:5050
Press CTRL+C to quit
192.168.0.92 - - [20/Dec/2024 19:24:01] "POST /api/user HTTP/1.1" 200 -
